# Langchain crash course

In [0]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key

# import os
# os.environ['OPENAI_API_KEY'] = "add your key here"

## LLMs

In [0]:
%pip install openai==0.28

  Obtaining dependency information for openai==0.28 from https://files.pythonhosted.org/packages/ae/59/911d6e5f1d7514d79c527067643376cddcf4cb8d1728e599b3b03ab51c69/openai-0.28.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/76.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.35.3
    Not uninstalling openai at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-13262ef9-7c6b-4b15-845e-723154db27af
    Can't uninstall 'openai'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# pip freeze

In [0]:
import openai
from openai import OpenAIError

# Function to interact with OpenAI
def safe_llm_call(prompt, model="gpt-3.5-turbo", temperature=0.9, max_tokens=100):
    try:
        # Calculate token usage estimation for prompt
        max_allowed_tokens = 4096  # ChatGPT model token limit
        prompt_tokens = len(prompt.split())  # Rough estimation: each word ~ 1 token
        available_response_tokens = max_allowed_tokens - prompt_tokens
        
        # Adjust max_tokens if prompt is too large
        if max_tokens > available_response_tokens:
            max_tokens = max(1, available_response_tokens)  # Ensure at least 1 token for response
        
        # Make the API call
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        return response['choices'][0]['message']['content']
    
    except OpenAIError as e:
        # Handle API errors
        print(f"OpenAI API error: {e}")
        return None
    except Exception as ex:
        # Handle any other unexpected errors
        print(f"Unexpected error: {ex}")
        return None

# Example usage
prompt = "Explain the principles of object-oriented programming."
response = safe_llm_call(prompt)
print(response)


OpenAI API error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
None


In [0]:
from langchain.llms import OpenAI
from openai import RateLimitError

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9, max_tokens=100)
try:
    name = llm.predict("I want to open a restaurant for Indian food. Suggest a fancy name for this.")
except RateLimitError as e:
    name = f"Error: {e}"
print(name)

Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


In [0]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\nSaffron & Spice.'

## Prompt Templates

In [0]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

# print(llm.predict(p))

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [0]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

'\n\n"La Casa de Sabor Mexicano"'

In [0]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new  chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


' \n\nTaco Tequila Palace'

In [0]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [0]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



- Chicken Tikka Masala
- Aloo Gobi
- Biryani
- Palak Paneer
- Chana Masala
- Samosas
- Naan
- Raita
- Korma
- Gulab Jamun
- Paneer Tikka
- Tandoori Chicken
- Vegetable Pakoras
- Mango Lassi
- Papadums


#### Sequential Chain

In [0]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [0]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [0]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [0]:
chain({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': "\n\nMaharaja's Spice Palace",
 'menu_items': '\n\n- Chicken Tikka Masala\n- Vegetable Biryani\n- Lamb Rogan Josh\n- Paneer Makhani\n- Aloo Gobi\n- Palak Paneer\n- Chana Masala\n- Vegetable Korma\n- Samosas\n- Naan Bread\n- Papadums\n- Raita\n- Mango Lassi'}

## Agents

In [0]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] = "add your serpapi key here"

#### serpapi and llm-math tool

In [0]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")



> Entering new  chain...
 I need to find the GDP of US in 2022
Action: Search
Action Input: US GDP in 2022
Observation: $25.46 trillion
Thought: I need to add 5 to this number
Action: Calculator
Action Input: 25.46 + 5
Observation: Answer: 30.46
Thought: I now know the final answer
Final Answer: The GDP of US in 2022 plus 5 is $30.46 trillion.

> Finished chain.


'The GDP of US in 2022 plus 5 is $30.46 trillion.'

#### Wikipedia and llm-math tool

In [0]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new  chain...
 I need to find out when Elon Musk was born and then calculate his age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a business magnate and investor. He is the founder, CEO, and chief engineer of SpaceX; angel investor, CEO, and product architect of Tesla, Inc.; owner, CTO, and chairman of Twitter; founder of the Boring Company and X Corp.; co-founder of Neuralink and OpenAI; and president of the philanthropic Musk Foundation. Musk is the wealthiest person in the world according to the Bloomberg Billionaires Index and Forbes's Real Time Billionaires list as of June 2023, primarily from his ownership stakes in Tesla and SpaceX, with an estimated net worth of around $225 billion according to Bloomberg and $230.2 billion according to Forbes.Musk was born in Pretoria, South Africa, and briefly attended the University of Pretoria before moving to Canada at age 18, acquiring citizen

'Elon Musk was born on June 28, 1971 and is currently 52 years old in 2023.'

## Memory

In [0]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [0]:
name = chain.run("Indian")
print(name)



Maharaja's Palace Cuisine


In [0]:
chain.memory

In [0]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [0]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [0]:
name = chain.run("Arabic")
print(name)



Al-Fez Restaurant


In [0]:
print(chain.memory.buffer)

Human: Mexican
AI: 

Taco Fiesta
Human: Arabic
AI: 

Al-Fez Restaurant


#### ConversationChain

In [0]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [0]:
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was won by the West Indies in 1975.'

In [0]:
convo.run("How much is 5+5?")

' 10.'

In [0]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.'

In [0]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first Cricket World Cup was won by the West Indies in 1975.
Human: How much is 5+5?
AI:  10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the West Indies team that won the first Cricket World Cup in 1975 was Clive Lloyd.


#### ConversationBufferWindowMemory

In [0]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was held in 1975 and the winner was the West Indies.'

In [0]:
convo.run("How much is 5+5?")

' 5+5 is 10.'

In [0]:
convo.run("Who was the captain of the winning team?")

" I'm sorry, I don't know."